In [136]:
import os
os.chdir("..")


In [155]:
import os
os.chdir("D:/VScode/Medical-Chatbot-using-Gen-AI-")

In [156]:
%pwd

'D:\\VScode\\Medical-Chatbot-using-Gen-AI-'

In [157]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [158]:

# Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [159]:
extract_data = load_pdf_file(data='Data/')

In [160]:
#extract_data

In [161]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [162]:
text_chunks = text_split(extract_data)
print(f"Total Chunks: {len(text_chunks)}")

Total Chunks: 5694


In [163]:
from langchain.embeddings import HuggingFaceEmbeddings

# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
    )
    return embeddings


In [164]:
embeddings= download_hugging_face_embeddings()

In [165]:
query_results = embeddings.embed_query("What is the capital of France?")
print(f"Length of Query Results: {len(query_results)}")

Length of Query Results: 384


In [166]:
#from dotenv import load_dotenv
# Load Environment Variables
#load_dotenv()

In [167]:
#PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")

In [168]:
from dotenv import load_dotenv
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Load variables from .env file
load_dotenv()

# Fetch API key from environment
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone
pc = Pinecone(api_key= PINECONE_API_KEY)

# Define index name and spec
index_name = "medicalbot1"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': 'd1b663d5918b0620542b6a23a9818dc2', 'date': 'Wed, 28 May 2025 19:32:07 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:
#Pinecone internally looks for the PINECONE_API_KEY in the environment variables
# Set the PINECONE_API_KEY environment variable
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'x-cloud-trace-context': '32d8f39e475e884432e5cd03e7b9d2d0', 'date': 'Wed, 28 May 2025 19:30:00 GMT', 'content-type': 'text/html', 'server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [ ]:
docsearch

In [ ]:
retrive = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3
    }
)

In [ ]:
retrived_docs= retrive.invoke("what is Cerebral palsy?")

In [ ]:
retrived_docs

[]

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the .env file

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
# Create the question-answering chain using the LLM and the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retrive, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input":"What is Cerebral palsy?"})
print(response["answer"])



Cerebral palsy is a group of disorders that affect movement and muscle coordination. It is caused by damage to the brain during development, often before or during birth. Symptoms can range from mild to severe and may include difficulty with motor skills, muscle stiffness, and trouble with speech or swallowing.


In [ ]:
response = rag_chain.invoke({"input":"What is the price of Tesla stock today?"})
print(response["answer"])



I don't know.
